<a href="https://colab.research.google.com/github/elhaithamy/elhaithamy.github.io/blob/master/Catgories_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import streamlit as st

# Expected headers
EXPECTED_COLUMNS = [
    "store", "ItemLookupCode", "Department", "Description", "Category",
    "SubCategroy", "TYPE", "SubType", "month", "n_order", "n_picked",
    "QTY", "average_ordered_qty", "total_amount", "n_dismissed", "total_loss_amount"
]

st.title("📊 Category & Item Level Analysis Tool")

# Upload Excel file
uploaded_file = st.file_uploader("Upload your Excel file", type=["xlsx"])

if uploaded_file:
    df = pd.read_excel(uploaded_file)

    # Validate headers
    if not all(col in df.columns for col in EXPECTED_COLUMNS):
        st.error("❌ Uploaded file must contain all required columns:")
        st.write(EXPECTED_COLUMNS)
    else:
        st.success("✅ File uploaded successfully and headers validated!")

        # --- FILTERS ---
        st.sidebar.header("Filters")

        # Store filter
        stores = df["store"].unique().tolist()
        selected_store = st.sidebar.multiselect("Select Store(s)", options=stores, default=stores)

        # Month filter
        months = df["month"].unique().tolist()
        selected_month = st.sidebar.multiselect("Select Month(s)", options=months, default=months)

        # Apply filters
        filtered_df = df[(df["store"].isin(selected_store)) & (df["month"].isin(selected_month))]

        st.sidebar.header("Choose Analysis")
        option = st.sidebar.radio(
            "Select analysis type:",
            (
                "Top-Selling Categories",
                "Low-Performing Categories",
                "Declining Sub-Categories",
                "Hero vs Dead SKUs",
                "Category Contribution",
                "ABC Classification",
                "Dismissal & Loss Analysis"
            )
        )

        # --- ANALYSIS OPTIONS ---
        if option == "Top-Selling Categories":
            result = filtered_df.groupby("Category")["total_amount"].sum().sort_values(ascending=False).head(10)
            st.subheader("🔥 Top-Selling Categories")
            st.dataframe(result)
            st.download_button("⬇️ Download Results", result.to_csv().encode("utf-8"), "top_selling.csv", "text/csv")

        elif option == "Low-Performing Categories":
            result = filtered_df.groupby("Category")["total_amount"].sum().sort_values(ascending=True).head(10)
            st.subheader("📉 Low-Performing Categories")
            st.dataframe(result)
            st.download_button("⬇️ Download Results", result.to_csv().encode("utf-8"), "low_performing.csv", "text/csv")

        elif option == "Declining Sub-Categories":
            subcat_month = filtered_df.groupby(["Category", "SubCategroy", "month"])["total_amount"].sum().reset_index()
            subcat_month = subcat_month.sort_values(["Category", "SubCategroy", "month"])
            subcat_month["pct_change"] = subcat_month.groupby(["Category", "SubCategroy"])["total_amount"].pct_change()
            declining = subcat_month[subcat_month["pct_change"] < 0]
            st.subheader("🔻 Declining Sub-Categories")
            st.dataframe(declining)
            st.download_button("⬇️ Download Results", declining.to_csv().encode("utf-8"), "declining_subcategories.csv", "text/csv")

        elif option == "Hero vs Dead SKUs":
            sku_sales = filtered_df.groupby(["ItemLookupCode", "Description"])["total_amount"].sum().reset_index()
            hero = sku_sales.sort_values("total_amount", ascending=False).head(10)
            dead = sku_sales.sort_values("total_amount", ascending=True).head(10)
            st.subheader("⭐ Hero SKUs")
            st.dataframe(hero)
            st.download_button("⬇️ Download Hero SKUs", hero.to_csv().encode("utf-8"), "hero_skus.csv", "text/csv")
            st.subheader("💀 Dead SKUs")
            st.dataframe(dead)
            st.download_button("⬇️ Download Dead SKUs", dead.to_csv().encode("utf-8"), "dead_skus.csv", "text/csv")

        elif option == "Category Contribution":
            contribution = filtered_df.groupby("Category")["total_amount"].sum()
            contribution = (contribution / contribution.sum() * 100).reset_index()
            contribution.columns = ["Category", "Contribution %"]
            st.subheader("📊 Category Contribution")
            st.dataframe(contribution)
            st.download_button("⬇️ Download Results", contribution.to_csv().encode("utf-8"), "category_contribution.csv", "text/csv")

        elif option == "ABC Classification":
            sku_sales = filtered_df.groupby(["ItemLookupCode", "Description"])["total_amount"].sum().sort_values(ascending=False).reset_index()
            sku_sales["cum_pct"] = sku_sales["total_amount"].cumsum() / sku_sales["total_amount"].sum()
            sku_sales["Class"] = pd.cut(sku_sales["cum_pct"], bins=[0, 0.8, 0.95, 1], labels=["A", "B", "C"])
            st.subheader("🔠 ABC Classification of SKUs")
            st.dataframe(sku_sales)
            st.download_button("⬇️ Download Results", sku_sales.to_csv().encode("utf-8"), "abc_classification.csv", "text/csv")

        elif option == "Dismissal & Loss Analysis":
            loss = filtered_df.groupby("Category")[["n_dismissed", "total_loss_amount"]].sum().reset_index()
            st.subheader("💸 Dismissal & Loss Analysis")
            st.dataframe(loss)
            st.download_button("⬇️ Download Results", loss.to_csv().encode("utf-8"), "loss_analysis.csv", "text/csv")


ModuleNotFoundError: No module named 'streamlit'

In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 490.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00
